# 3D Image gradient at presence of noise
***Mar 4 , 2022***

In MRI data, a common noise type is the [Rician noise](https://doi.org/10.1002/mrm.1910340618). As explained in [this article](https://doi.org/10.1002/cmr.a.20124), the Rician noise is in essence the result of Gaussian noise on both the real and imaginary channel of complex data, and when calculating the magnitude, the magnitude follows a PDF which is known as the Rician distribution.

In this notebook the effects of four types of noises common in MRI images on the sobel operator are explored:
- Rician
- Gaussian
- K space spike
- Gibbs

#### _load 3D MRI image data_

In [ ]:
import os
import math

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import scipy, scipy.io
from scipy import ndimage, signal

import monai
import h5py

In [ ]:
# data file name
fname = '101915_3T_T2w_SPC1'
fnamemix = '105216_3T_T2w_SPC2'
# Load data
with h5py.File(os.path.join('data', fname+'.h5')) as f:
    x = f['data'][()]
    y = f['target'][()]
# print('Shape of x (input, Low Res) =', x.shape) # (128, 128, 32)
# print('Shape of y (target, High Res) =', y.shape) # (256, 256, 64)

In [ ]:
def visualise_img(image, fig_size=(4, 8), res=500):
    fig = plt.figure(dpi=res)
    grid = ImageGrid(fig, 111, nrows_ncols=fig_size, axes_pad=0.0)
    for i in range(image.shape[-1]):
        #grid[i].imshow(np.rot90(image[i, ...], 1), cmap='gray') # if rot90 here, the translatory transform direction changed
        grid[i].imshow(image[..., i], cmap='gray')
        grid[i].axis('off')
        grid[i].set_xticks([])
        grid[i].set_yticks([])
    plt.show()

In [ ]:
visualise_img(x, (4,8))

## 1. apply noise to data
### Rician

In [ ]:
noise_rician = monai.transforms.RandRicianNoise(prob=1.0, std=0.8, relative=True, channel_wise=True, sample_std=False)
x_rician = noise_rician(x)
visualise_img(x_rician, (4,8))

### Gaussian

In [ ]:
noise_gaussian = monai.transforms.RandGaussianNoise(prob=1.0, mean=0.0, std=800)
x_gaussian = noise_gaussian(x)
visualise_img(x_gaussian, (4,8))

### K space spike

In [ ]:
loc = (60,0)
noise_kspacespike = monai.transforms.KSpaceSpikeNoise(loc, k_intensity=16, as_tensor_output=True)
x_kspacespike = noise_kspacespike(x)
visualise_img(x_kspacespike, (4,8))

### Gibbs

In [ ]:
noise_gibbs = monai.transforms.GibbsNoise(alpha=0.8, as_tensor_output=True)
x_gibbs = noise_gibbs(x)
visualise_img(x_gibbs, (4,8))

## 2. apply image gradient operator (sobel)

In [ ]:
sobel_x = ndimage.sobel(x, axis=1)
visualise_img(sobel_x, (4, 8))

In [ ]:
sobel_x_rician = ndimage.sobel(x_rician, axis=1)
visualise_img(sobel_x_rician, (4, 8))

In [ ]:
sobel_x_gaussian = ndimage.sobel(x_gaussian, axis=1)
visualise_img(sobel_x_gaussian, (4, 8))

In [ ]:
sobel_x_kspacespike = ndimage.sobel(x_kspacespike, axis=1)
visualise_img(sobel_x_kspacespike, (4, 8))

In [ ]:
sobel_x_gibbs = ndimage.sobel(x_gibbs, axis=0)
visualise_img(sobel_x_gibbs, (4, 8))

## 3. Discussion

- The image data contaminated by rician and gaussian noise shows a 'white noise' feature in both raw image and the gradient image. 
- Surprisingly the K space spike noise applied is the most distorted visually but least affected sobel-wise.
- the Gibbs noise produces a 'ringing artifacts' observable in both the raw and gradient images

The comparison across these noises are not significant not only because their power or intensity can not be compared in one scale, but also the physical source producing these noises can not be compared and scaled.

### *About the explanation of the minimal effect of K space spike to sobel*

Fourier-wise speaking, the sobel operator can be viewed as a high-pass filter, which detects the fast changing components(gradients) of the 2D signal. The K space spike applied in the example above is close to the center (60 in 128) and is a low frequency signal, which shall be less significant (or suppressed in views of the frequency spectrum) by the sobel operator generally in high SNR areas. At low SNR areas the noise itself is the primary source of change and can be visually seen in gradient images due to the normalization of the plt.imshow() function.

To justify the theory above, another K spike noise with a higher frequency is applied below.

In [ ]:
loc = (25,0)
noise_kspacespike_2 = monai.transforms.KSpaceSpikeNoise(loc, k_intensity=16, as_tensor_output=True)
x_kspacespike_2 = noise_kspacespike_2(x)
visualise_img(x_kspacespike_2, (4,8))

In [ ]:
sobel_x_kspacespike_2 = ndimage.sobel(x_kspacespike_2, axis=1)
visualise_img(sobel_x_kspacespike_2, (4, 8))

### *The K space image (FFT, log)*

In [ ]:
x_ft = scipy.fft.fft2(x, axes=(0,1)) # dtype('complex128')
visualise_img(np.log(np.abs(x_ft)), (4,8))

In [ ]:
sobel_x_ft = scipy.fft.fft2(sobel_x, axes=(0,1))
visualise_img(np.log(np.abs(sobel_x_ft)), (4,8))

Since the scale of the raw fft and sobel fft are not the same due to plt.imshow() normalization, the two data should be plotted in one single pic:

In [ ]:
compare_img = np.concatenate((x_ft[...,0], sobel_x_ft[...,0]), axis=1)
plt.imshow(np.log(np.abs(compare_img)), cmap='gray')
plt.show()

take the difference of the two as the sobel masking effect in k space:

In [ ]:
sobel_x_mask = x_ft[...,0] - sobel_x_ft[...,0]
plt.imshow(np.log(np.abs(sobel_x_mask)), cmap='gray')
plt.show()

do the same analysis to the spike noise contaminated data:

In [ ]:
x_kspacespike_2_ft = scipy.fft.fft2(x_kspacespike_2, axes=(0,1))
visualise_img(np.log(np.abs(x_kspacespike_2_ft)), (4,8))

In [ ]:
sobel_x_kspacespike_2_ft = scipy.fft.fft2(sobel_x_kspacespike_2, axes=(0,1))
visualise_img(np.log(np.abs(sobel_x_kspacespike_2_ft)), (4,8))

### *the maskings*:

In [ ]:
sobel_x_kspacespike_2_mask = x_kspacespike_2_ft - sobel_x_kspacespike_2_ft
visualise_img(np.log(np.abs(sobel_x_kspacespike_2_mask)), (4,8))

the noise spike line shows clearly on the masking image; the noise are being filtered out!

### *inverse fft* of the maskings:

In [ ]:
sobel_x_kspacespike_2_mask_ifft = scipy.fft.ifft2(sobel_x_kspacespike_2_mask, axes=(0,1))
visualise_img(np.log(np.abs(sobel_x_kspacespike_2_mask_ifft)), (4,8))